In [1]:
import os, geopandas as gpd
#Importaremos cada archivo "shp" en un caracter (paises, rios y ciudades)
paises= gpd.read_file("https://github.com/betsfv/introgeodf/raw/refs/heads/main/maps/World_Countries/World_Countries.shp")
rios =gpd.read_file("https://github.com/betsfv/introgeodf/raw/refs/heads/main/maps/World_Hydrography/World_Hydrography.shp")
ciudades=gpd.read_file("https://github.com/betsfv/introgeodf/raw/refs/heads/main/maps/World_Cities/World_Cities.shp")

ImportError: The 'read_file' function requires the 'pyogrio' or 'fiona' package, but neither is installed or imports correctly.
Importing fiona resulted in: DLL load failed while importing _env: No se encontró el proceso especificado.
Importing pyogrio resulted in: No module named 'pyogrio'